# Использование метода ближайших соседей для решения задачи классификации
Группа: ИВТ-М20.<br/>
Студент: Архипов Д.А.

In [13]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib as mpl
import matplotlib.pyplot as plt

from pandas import (
    DataFrame,
    read_csv
)
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [14]:
plt.style.use(['ggplot'])

## Чтение данных из файлов

In [5]:
target = DataFrame(read_csv('data/target1.csv', header=0))
target.columns = ['data']
var = DataFrame(read_csv('data/var1.csv', sep=',', header=None))


## Транспонирование исходной матрицы
## Отображение нескольких записей

In [6]:
targetT = target.T
varT = var.T
print(targetT.head())
print(targetT.tail())
print(varT.head())
print(varT.tail())

      0   1   2   3   4   5   6   7   8   9   ...  50  51  52  53  54  55  56  \
data   1   1   1   1   1   1   1   1   1   1  ...   2   2   2   2   2   2   2   

      57  58  59  
data   2   2   2  

[1 rows x 60 columns]
      0   1   2   3   4   5   6   7   8   9   ...  50  51  52  53  54  55  56  \
data   1   1   1   1   1   1   1   1   1   1  ...   2   2   2   2   2   2   2   

      57  58  59  
data   2   2   2  

[1 rows x 60 columns]
          0         1         2         3         4          5         6   \
0  10.660937  9.266645  8.163384  9.291998  6.683817  11.054668  9.368904   
1   8.837976  8.422462  7.887574  8.345648  6.691520   9.338379  8.224688   
2   4.869347  5.690374  4.825932  5.327111  5.066057   4.523733  3.887042   
3   3.841726  5.288656  6.759585  5.093615  3.972489   6.223527  5.677149   
4   2.965282  5.558424  7.383261  5.726576  3.536099   6.199316  4.605147   

          7         8         9   ...         50         51         52  \
0  12.977878  9

## Обучающаяся и проверочная выборка

In [7]:
print(target.shape, var.shape)
target_train, target_test, var_train, var_test = train_test_split(target, var, test_size=0.3, train_size=0.7, shuffle=0)
print(target_test.shape, var_test.shape)
print(target_train.shape, var_train.shape)

(60, 1) (60, 200)
(18, 1) (18, 200)
(42, 1) (42, 200)


## Алгоритм K-ближайших соседей

In [8]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(var_train, target_train)
scores = cross_val_score(knn, var_train, target_train, cv=10, scoring='accuracy')
print(f"Test score {scores}")

knn.predict(var_test)

Test score [1.   1.   1.   1.   1.   1.   1.   0.75 1.   1.  ]


array([2, 1, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 1], dtype=int64)

## Оценка качества

In [9]:
y_true = target_test
y_pred = knn.predict(var_test)

print(accuracy_score(y_true, y_pred, normalize=True))
print(classification_report(y_true, y_pred))

0.9444444444444444
              precision    recall  f1-score   support

           1       0.88      1.00      0.93         7
           2       1.00      0.91      0.95        11

    accuracy                           0.94        18
   macro avg       0.94      0.95      0.94        18
weighted avg       0.95      0.94      0.94        18



## Подбор гиперпараметров

In [10]:
parameters = { 'n_neighbors': [3,5,11,19],
               'weights': ['uniform', 'distance'],
               'metric': ['euclidean', 'manhattan']}
grid = GridSearchCV(KNeighborsClassifier(),
                    parameters,
                    cv=5)
gs_result = grid.fit(var_train, target_train)

## Повторное обучение

In [11]:
print(gs_result.best_params_)
print(gs_result.best_score_)
print(gs_result.predict(var_test))

y_pred = gs_result.predict(var_test)
print(accuracy_score(y_true, y_pred, normalize=True))

{'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'uniform'}
0.975
[2 1 2 2 1 1 1 2 2 2 2 2 1 2 1 1 2 1]
0.9444444444444444


# Ответы на контрольные вопросы

## 1. Опишите этапы реализации алгоритма KNN и для решения каких задач его можно использовать?
### Этапы реализации
Алгоритм K-ближайших соседей (KNN) использует «сходство признаков» для прогнозирования значений новых точек данных, что также означает, что новой точке данных будет присвоено значение на основе того, насколько близко он соответствует точкам в обучающем наборе. Мы можем понять его работу с помощью следующих шагов —

Шаг 1 — Для реализации любого алгоритма нам нужен набор данных. Таким образом, во время первого шага KNN мы должны загрузить данные обучения, а также данные испытаний.

Шаг 2 — Далее нам нужно выбрать значение K, то есть ближайшие точки данных. K может быть любым целым числом.

Шаг 3 — Для каждой точки в тестовых данных сделайте следующее —

3.1 — Рассчитайте расстояние между данными испытаний и каждой строкой данных тренировки с помощью любого из методов, а именно: Евклидово, Манхэттенское или Хэмминговское расстояние. Наиболее часто используемый метод расчета расстояния — евклидов.

3.2 — Теперь, основываясь на значении расстояния, отсортируйте их в порядке возрастания.

3.3 — Далее он выберет верхние K строк из отсортированного массива.

3.4 — Теперь он назначит класс контрольной точке на основе наиболее часто встречающегося класса этих строк.

3.1 — Рассчитайте расстояние между данными испытаний и каждой строкой данных тренировки с помощью любого из методов, а именно: Евклидово, Манхэттенское или Хэмминговское расстояние. Наиболее часто используемый метод расчета расстояния — евклидов.

3.2 — Теперь, основываясь на значении расстояния, отсортируйте их в порядке возрастания.

3.3 — Далее он выберет верхние K строк из отсортированного массива.

3.4 — Теперь он назначит класс контрольной точке на основе наиболее часто встречающегося класса этих строк.

Шаг 4 — Конец
### Области применения алгоритма KNN
Алгоритм k-ближайших соседей имеет широкое применение. Например:

- Обнаружение мошенничества. Новые случаи мошенничества могут быть похожи на те, которые происходили когда-то в прошлом. Алгоритм KNN может распознать их для дальнейшего рассмотрения.
- Предсказание отклика клиентов. Можно определить отклик новых клиентов по данным из прошлого.
- Медицина. Алгоритм может классифицировать пациентов по разным показателям, основываясь на данных прошедших периодов.
- Прочие задачи, требующие классификацию.

## 2. Какие метрики используются для оценки качества работы алгоритмов классификации? Опишите данные метрики с математической точки зрения и скажите, чем они отличаются друг от друга.
[Подробно](https://goo-gl.ru/MlQae "Оценка качества")

## 3. Каким способом можно повысить качество работы алгоритмов классификации?
